### Fair Clustering Through Fairlets

Paper Link: https://proceedings.neurips.cc/paper_files/paper/2017/hash/978fce5bcc4eccc88ad48ce3914124a2-Abstract.html

Balance: min(#RED(Y) / #BLUE(Y) , #BLUE(Y) / #RED(Y))
value is between 0 and 1

balance(C) <= balance(X)
where C is a clustering
X is number of all points

(1,1)-fairlets
Create bipartite graph between B and R (blue and red) with edges between any bichromatic pair of nodes
cost of this graph: max weight of the edges

define T which is a threshold max weight.
The graph with the smallest T has the perfect matching.
Can be found by binary search.

(1,t')-fairlets
works for balance t <= 1.
t = 1/t' for some int t'
transform into MCF-problem
T is a parameter of the algorithm
Create directed graph HT=(V,E)
Its node set is composed of 2 special nodes beta, p and all nodes from B and R and t' additional copies of the nodes in B and R. 

The directed edges of H:
* (beta, p) edge with cost 0 and capacity min(|B|,|R|)
* (beta, bi) edge for each bi in B with cost 0 and capacity t'-1
* (ri, p) edge for each ri in R with cost 0 and capacity t'-1
* an edge for each bi in B to each of its t' copies with cost 0 and capacity 1
* an edge for each ri in R to each of its t' copies with cost 0 and capacity 1
* for each bi in B, rj in R and  1 <= k,l <= t' an edge (bik, rjl) wtih capacity 1. The cost of this edge is 1 if d(bi, rj) <= T, else infinite

nodes of H:
* each node in B has supply of 1 
* each node in R has demand of 1
* beta has supply of |R| -->
* p has demand of |B| -->
* every other node has demand and supply of 0

Steps:
1. Create adjacency matrix according to the rules above
2. Calculate MCF with the matrix
3. 

In [19]:
#imports
import pandas as pd
import numpy as np
import json
from pyclustering.cluster.kmedians import kmedians

### Load and prepare datasets

In [3]:
from ucimlrepo import fetch_ucirepo 

dataset_name = "adult"
randomstate = 42

#get config file
with open('datasetconfig.json') as config_file:
    config = json.load(config_file)

# fetch dataset
dataset = None
if(dataset_name == "adult"):
    dataset = fetch_ucirepo(id=2) 
elif(dataset_name == "bank"):
    dataset = fetch_ucirepo(id=222)

X = dataset.data.features
y = dataset.data.targets

data = X
data = data[data[config[dataset_name]['sensitive_column']].isin(config[dataset_name]['sensitive_values'])]

for sensitive_value in config[dataset_name]['sensitive_values']:
    print("\nAmount of %s values:  %d" %(sensitive_value, data.value_counts(config[dataset_name]['sensitive_column'])[sensitive_value]))

#only choose the sensitive column and the columns used for distance measuring mentioned in the paper
#the rest of the attributes seem to be discarded
data = data[[config[dataset_name]['sensitive_column']] + config[dataset_name]['distance_columns']].copy()
data[config[dataset_name]['sensitive_column']] = np.where(data[config[dataset_name]['sensitive_column']] == config[dataset_name]['sensitive_values'][0], 0, 1)

#choose a random sample based on the sample size mentioned in the paper
sample = data.sample(n=config[dataset_name]['subset_size'], random_state=randomstate)
sample.reset_index(drop=True)

#set the values for male and female to 0 and 1 respectively and add their indices to their respective lists
sample_reds = list(sample[sample[config[dataset_name]['sensitive_column']]==0].index)
sample_blues = list(sample[sample[config[dataset_name]['sensitive_column']]==1].index)



Amount of Male values:  32650

Amount of Female values:  16192


In [4]:
def get_balance(data, sensitive_column):
    return min(data.value_counts(sensitive_column)[0]/data.value_counts(sensitive_column)[1],data.value_counts(sensitive_column)[1]/data.value_counts(sensitive_column)[0])

In [5]:
#output some information about the chosen sample
sample_balance = get_balance(sample, config[dataset_name]['sensitive_column'])
print("balance of the chosen sample: " + str(sample_balance))

red_count = len(sample_reds)
blue_count = len(sample_blues)

print("red count: " + str(red_count))
print("blue count: " + str(blue_count))

balance of the chosen sample: 0.5306122448979592
red count: 392
blue count: 208


In [6]:
import math

#compute euclidean distance
def compute_distance(a, b, dataset_name):
    res = 0
    for distance_column in config[dataset_name]['distance_columns']:
        res += (a[distance_column] - b[distance_column]) ** 2

    return math.sqrt(res)

#returns a 2-dimensional array of distances between all red and blue points
def get_distances(a, b):

    distances = [[0]* len(b)] * len(a)

    for idx_blue, i in enumerate(a):
        for idx_red, j in enumerate(b):
            distances[idx_blue][idx_red] = compute_distance(sample.loc[i], sample.loc[j], dataset_name=dataset_name)

    return distances

distances = get_distances(sample_blues, sample_reds)

In [10]:
print(len(distances))

208


### implementaion of the (t,k)-fairlet approximation

In [11]:
import networkx as nx

def create_MCF(distances, clustering_method="k-centers", t=2, T=400, maxCost=1000000):
    #supply = negative demand
    #cost = weight

    G = nx.DiGraph()
    #add special nodes beta and rho and an edge between them
    G.add_node('beta', demand=(-1*red_count))
    G.add_node('rho', demand=blue_count)
    G.add_edge('beta','rho', weight=0, capacity=min(red_count,blue_count))

    #create a node for each b and r
    for i in range(blue_count):
        G.add_node('b%d'%(i+1), demand=-1)
        G.add_edge('beta','b%d'%(i+1), weight=0, capacity=t-1)
    for i in range(red_count):
        G.add_node('r%d'%(i+1), demand=1)
        G.add_edge('r%d'%(i+1), 'rho', weight=0, capacity=t-1)


    #create t' copies of the b and r nodes
    for i in range(blue_count):
        for extra_node_count in range(t):
            G.add_node('b%d_%d'%(i+1, extra_node_count+1), demand=0)
            G.add_edge('b%d'%(i+1),'b%d_%d'%(i+1,extra_node_count+1), weight=0, capacity=1)
    for i in range(red_count):
        for extra_node_count in range(t):
            G.add_node('%d_%d'%(i+1, extra_node_count+1), demand=0)
            G.add_edge('r%d_%d'%(i+1, extra_node_count+1), 'r%d'%(i+1), weight=0, capacity=1)

    #add edges between the t' additional b and r nodes
    for i in range(blue_count):
        for k in range(t):
            for j in range(red_count):
                for l in range(t):
                    distance = distances[i][j]
                    if(distance <= T):
                        if(clustering_method == "k-centers"):
                            G.add_edge('b%d_%d'%(i+1, k+1), 'r%d_%d'%(j+1, l+1), weight=1, capacity=1)
                        elif(clustering_method == "k-medians"):
                            G.add_edge('b%d_%d'%(i+1, k+1), 'r%d_%d'%(j+1, l+1), weight=distance, capacity=1)
                    else: 
                        G.add_edge('b%d_%d'%(i+1, k+1), 'r%d_%d'%(j+1, l+1), weight=maxCost, capacity=1)

    return G


In [12]:
G = create_MCF(distances)
flowCost, flowDictionary = nx.network_simplex(G)

In [13]:
fairlets = []

for dictKey in flowDictionary.keys():
    if "b" in dictKey and "_" in dictKey:
        if sum(flowDictionary[dictKey].values()) >= 1:
            for r_dictKey in flowDictionary[dictKey].keys():
                if flowDictionary[dictKey][r_dictKey] == 1:
                    if not any(dictKey.split('_')[0] in d['blues'] for d in fairlets)  and not any(r_dictKey.split('_')[0] in d['reds'] for d in fairlets):
                        fairlets.append({'blues': [dictKey.split('_')[0]], 'reds': [r_dictKey.split('_')[0]]})
                    elif any(dictKey.split('_')[0] in d['blues'] for d in fairlets)  and not any(r_dictKey.split('_')[0] in d['reds'] for d in fairlets):
                        for fairlet in fairlets:
                            if dictKey.split('_')[0] in fairlet['blues']:
                                fairlet['reds'].append(r_dictKey.split('_')[0])
                    elif not any(dictKey.split('_')[0] in d['blues'] for d in fairlets)  and any(r_dictKey.split('_')[0] in d['reds'] for d in fairlets):
                        for fairlet in fairlets:
                            if r_dictKey.split('_')[0] in fairlet['reds']:
                                fairlet['blues'].append(dictKey.split('_')[0])

fairlets

[{'blues': ['b1'], 'reds': ['r392', 'r391']},
 {'blues': ['b2'], 'reds': ['r390', 'r389']},
 {'blues': ['b3'], 'reds': ['r388', 'r387']},
 {'blues': ['b4'], 'reds': ['r386', 'r385']},
 {'blues': ['b5'], 'reds': ['r384', 'r383']},
 {'blues': ['b6'], 'reds': ['r382', 'r381']},
 {'blues': ['b7'], 'reds': ['r380', 'r379']},
 {'blues': ['b8'], 'reds': ['r378', 'r377']},
 {'blues': ['b9'], 'reds': ['r376', 'r375']},
 {'blues': ['b10'], 'reds': ['r374', 'r373']},
 {'blues': ['b11'], 'reds': ['r372', 'r371']},
 {'blues': ['b12'], 'reds': ['r370', 'r369']},
 {'blues': ['b13'], 'reds': ['r368', 'r367']},
 {'blues': ['b14'], 'reds': ['r366', 'r365']},
 {'blues': ['b15'], 'reds': ['r364', 'r363']},
 {'blues': ['b16'], 'reds': ['r362', 'r361']},
 {'blues': ['b17'], 'reds': ['r360', 'r359']},
 {'blues': ['b18'], 'reds': ['r358', 'r357']},
 {'blues': ['b19'], 'reds': ['r356', 'r355']},
 {'blues': ['b20'], 'reds': ['r354', 'r353']},
 {'blues': ['b21'], 'reds': ['r352', 'r351']},
 {'blues': ['b22'], 'r

In [30]:
fairlet_information = []
fairlet_centers = []
fairlet_costs = []

for fairlet in fairlets:
    fairlet_distances = {}
    distances = []
    for blue in fairlet['blues']:
        for blue2 in fairlet['blues']:
            if blue != blue2:
                distances.append(compute_distance(sample.loc[sample_blues[int(blue[1:])-1]], sample.loc[sample_blues[int(blue2[1:])-1]], dataset_name=dataset_name))
        for red in fairlet['reds']:
            distances.append(compute_distance(sample.loc[sample_blues[int(blue[1:])-1]], sample.loc[sample_reds[int(red[1:])-1]], dataset_name=dataset_name))
        fairlet_distances[blue] = max(distances)
        distances = []

    for red in fairlet['reds']:
        for blue in fairlet['blues']:
            distances.append(compute_distance(sample.loc[sample_reds[int(red[1:])-1]], sample.loc[sample_blues[int(blue[1:])-1]], dataset_name=dataset_name))
        for red2 in fairlet['reds']:
            if red != red2:
                distances.append(compute_distance(sample.loc[sample_reds[int(red[1:])-1]], sample.loc[sample_reds[int(red2[1:])-1]], dataset_name=dataset_name))
        fairlet_distances[red] = max(distances)

    center = min(fairlet_distances, key=fairlet_distances.get)
    fairlet_centers.append(center)
    fairlet_costs.append(fairlet_distances[center])
    fairlet_information.append(fairlet_distances)
    print(fairlet_distances)
print(len(fairlet_information))

{'b1': 131244.0016800768, 'r392': 117344.00394140299, 'r391': 131244.0016800768}
{'b2': 484852.0001165304, 'r390': 523360.0001050902, 'r389': 523360.0001050902}
{'b3': 162593.50677379462, 'r388': 83933.00074464155, 'r387': 162593.50677379462}
{'b4': 24968.291631587454, 'r386': 47016.81435401594, 'r385': 47016.81435401594}
{'b5': 155499.003549862, 'r384': 121331.00280225166, 'r383': 155499.003549862}
{'b6': 95481.00434641437, 'r382': 80053.45093123718, 'r381': 95481.00434641437}
{'b7': 74398.0027151267, 'r380': 74398.0027151267, 'r379': 74398.0027151267}
{'b8': 79958.00355186465, 'r378': 155281.00196418105, 'r377': 155281.00196418105}
{'b9': 149996.00550014657, 'r376': 165705.00234452792, 'r375': 165705.00234452792}
{'b10': 155203.00082794792, 'r374': 113462.80760672195, 'r373': 155203.00082794792}
{'b11': 362748.0000716751, 'r372': 324633.0010427159, 'r371': 362748.0000716751}
{'b12': 285395.00130696053, 'r370': 185183.00228152692, 'r369': 285395.00130696053}
{'b13': 214467.00033571597

In [97]:
n_clusters = 5
medians = sample.to_numpy()[np.random.choice(range(0, sample.shape[0]), size=n_clusters, replace=False)]

#unfair kmedians
kmedians_vanilla = kmedians(sample, medians)
kmedians_vanilla.process()
kmedians_vanilla.get_clusters()

#fair kmedians
sample_fairlet_centers = []
index = 0
for fairlet_center in fairlet_centers:
    if(fairlet_center[:1] == 'r'):
        sample_fairlet_centers.append(sample.loc[sample_reds[int(fairlet_center[1:])-1]].drop("sex").to_list())
    else:
        sample_fairlet_centers.append(sample.loc[sample_blues[int(fairlet_center[1:])-1]].drop("sex").to_list())

sample_fairlet_centers_np = np.array(sample_fairlet_centers)
fairlet_medians = sample_fairlet_centers_np[np.random.choice(range(0, sample_fairlet_centers_np.shape[0]), size=n_clusters, replace=False)]
kmedians_fairlets = kmedians(sample_fairlet_centers, fairlet_medians)
kmedians_fairlets.process()
fairlet_clusters = kmedians_fairlets.get_clusters()

In [98]:
full_fairlet_clustering = []

for fairlet_cluster in fairlet_clusters:
    cluster_points = []
    for fairlet_center in fairlet_cluster:
        fairlet = fairlets[fairlet_center]
        blues = fairlet['blues']
        reds = fairlet['reds']
        for blue in blues:
            cluster_points.append(sample.loc[sample_blues[int(blue[1:])-1]])
        for red in reds:
            cluster_points.append(sample.loc[sample_reds[int(red[1:])-1]])
    
    full_fairlet_clustering.append(cluster_points)


In [ ]:
#TODO:
#nice graph comparison